In [1]:
!pip install transformers sentencepiece torch

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re


In [4]:
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)


In [5]:
def get_intent(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    scores = torch.softmax(outputs.logits, dim=1)
    positive = scores[0][1].item()

    # simple rule-based intent detection for purchase requests
    if "buy" in text.lower() or "purchase" in text.lower() or "quotation" in text.lower():
        return "purchase_request"
    elif positive > 0.7:
        return "positive_sentiment"
    else:
        return "general_query"


In [6]:
def extract_requirements(text):
    product = None
    brand = None
    quantity = None
    details = None

    # Quantity extraction
    qty = re.search(r"\b(\d+)\b", text)
    if qty:
        quantity = qty.group(1)

    # List of common products
    products = ["laptop", "laptops", "printer", "monitor", "router"]
    for p in products:
        if p in text.lower():
            product = p
            break

    # Brands
    brands = ["lenovo", "dell", "hp", "asus", "acer"]
    for b in brands:
        if b in text.lower():
            brand = b
            break

    # Extra details
    specs = re.search(r"(8GB RAM|16GB RAM|i5|i7|\d+GB|\d+TB)", text, re.IGNORECASE)
    if specs:
        details = specs.group(1)

    return {
        "product": product,
        "brand": brand,
        "quantity": quantity,
        "details": details
    }


In [7]:
def process_message(text):
    return {
        "intent": get_intent(text),
        "requirements": extract_requirements(text)
    }


In [8]:
msg = "Hi team, I want to purchase 20 Lenovo laptops with 16GB RAM. Send quotation."
result = process_message(msg)
result


{'intent': 'purchase_request',
 'requirements': {'product': 'laptop',
  'brand': 'lenovo',
  'quantity': '20',
  'details': '16GB RAM'}}